In [55]:
import pandas as pd
UB = pd.read_csv('tailard_UB.csv')
UB = pd.DataFrame(UB)

In [56]:
from urllib.request import Request, urlopen
import urllib.parse
from bs4 import BeautifulSoup
import numpy as np
import requests

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"}

In [105]:
for i in range(0, 1):
    size_text = UB.iloc[i, 1]
    size_list = size_text.rsplit('x', 1)
    n_jobs = int(size_list[0])
    n_mchs = int(size_list[1])

    UB_temp = UB.iloc[i, 2]

    # i_2d = str(i+1).zfill(2)
    # url = 'https://optimizizer.com/solution.php?name=ta' + str(i_2d) + '&UB=' + str(UB_temp) + '&problemclass=ta' 

    # response = requests.get(url, headers=headers)
    # html = response.text
    # soup = BeautifulSoup(html, 'html.parser')

    # nums = soup.get_text().rsplit(':', 1)[-1].rsplit(' back', 1)[0]
    # table = list(map(int, nums.split(' ')))

    # table_opt = np.array(table).reshape(n_jobs, n_mchs)
    # print(table_opt)

    target_url = 'http://mistic.heig-vd.ch/taillard/problemes.dir/ordonnancement.dir/jobshop.dir/tai15_15.txt'
    response = requests.get(target_url)
    data = response.text
    soup2 = BeautifulSoup(data, 'html.parser')

    numbs = soup2.get_text().rsplit('Times', 1)[1].rsplit('Machines')[0]
    new_mch = numbs.replace('\r\n', '')[1:]
    
    table2 = list(map(int, new_mch.split()))

    table_dur = np.array(table2).reshape(n_jobs, n_mchs)
    print(table_dur)

The ith row of each solution file represents the processing order of jobs on the ith machine
(indexing of jobs starts from zero)

In [89]:
target_url = 'http://mistic.heig-vd.ch/taillard/problemes.dir/ordonnancement.dir/jobshop.dir/tai15_15.txt'
response = requests.get(target_url)
data = response.text
soup2 = BeautifulSoup(data, 'html.parser')

numbs = soup2.get_text().rsplit('Times', 1)[1].rsplit('Machines')[0]
new_mch = numbs.replace('\r\n', '')[1:]
new_mch

'35 78 79 65 53 14 93 70 14 90 95 49 36 85  1 83 41 22 29 52 71 16 93 54 63 12 85 62 45 30 60 43 71  2 50 37 86 81 60 57 66 24 98 92 69 14 59 35  6 25 57  1 44 94 30 95 93 51 52 16 96 39 75 98  2 38 69 32 95 63  4 11 50 95 78 73 28 43 47 57 88 33 13  7 49 23 38 21 99 72  3 80 67 93 91 31 52 64 83  2 90 64 16 18 25 23 30 22 54 68 63 89 95  5 37  5 42 17 54 46 44 59 87 62 51 55  3 40 26 18 15 18 72 35 60 27 14 77 24 55 67 59 19 29 33 88 30 91 11 11 67 94 50  2 83 19 29 37 58 32 38 99 88 49 70 60  7 81 82 58 83 16  1 69  7  3 84  8 12 93 92 81  4 78  9 78 75  5 50  8 44  4 60 94 74 32 88 31 68 31 10 45 75 82 51 55 99 44 84 22 12 35 64 17 42 46 65 74 96 28 86 95 93 67 56'